In [3]:
# 5-line fuzzer below is from Charlie Miller's
# "Babysitting an Army of Monkeys":
# Part 1 - http://www.youtube.com/watch?v=XnwodiCBws
# Part 2 - http://www.youtube.com/watch?v=lK5fgCvS2N4

import os
import fnmatch

working_dir = r"C:\fuzzer"
# Set working directory:
os.chdir(working_dir)
# Create a list of all the files in the working directory that end in .odt:
file_list = fnmatch.filter(os.listdir("."), "*.odt")

# Application to test: OpenOffice Writer, version 4.0.0
apps = [r"C:\Program Files (x86)\OpenOffice 4\program\swriter"]

FuzzFactor = 1000
num_tests = 15
fuzz_output = "fuzz.odt"

import math
import random
import subprocess
import time

for i in range(num_tests):
    file_choice = random.choice(file_list)
    app = random.choice(apps)
    with open(file_choice, 'rb') as f:
        buf = bytearray(f.read())
    
    # start Charlie Miller code 
    numwrites = random.randrange(math.ceil(len(buf) / float(FuzzFactor))) + 1
    for j in range(numwrites):
        rbyte = random.randrange(256)
        rn = random.randrange(len(buf))
        buf[rn] = "%c" % (rbyte)
    #end Charlie Miller code

    with open(fuzz_output, 'wb') as f:
        f.write(buf)
    
    log = open("log.txt", 'a+')
    
    # Create a 'Popen' process object
    process = subprocess.Popen([app, fuzz_output])
    # Wait a second...
    time.sleep(1)
    # Get the process 'returncode' attribute
    crashed = process.poll()
    if crashed is not None:
        # Run Windows 'taskkill' command, to forcefully terminate the application and any
        # child processes started by it. This is necessary as Popen.terminate() or Popen.kill()
        # will not work if a dialogue box is open, as is the case with OpenOffice, which asks
        # the user if he wants to allow OpenOffice to try and repair the file
        subprocess.call(['taskkill', '/F', '/T', '/PID', str(process.pid)])
        log.write(str(i) + '\t' + app + '\t' + file_choice + "\n")
    else:
        crash_file = 'crash/' + str(i) + '_' + app + '_' + file_choice
        print app, "crashed with file", crash_file
    
    time.sleep(1)

log.close()